# Install

In [1]:
!pip install -U lightautoml

# Library

In [2]:
import pandas as pd
from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML

# Load

In [3]:
train_data = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
train_data.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
0,0,-0.002350,59,0.766739,-1.350460,42.2727,16.68570,30.3599,1.267300,0.392007,...,-42.43990,26.854000,1.45751,0.696161,0.941764,1.828470,0.924090,2.29658,10.48980,15
1,1,0.784462,145,-0.463845,-0.530421,27324.9000,3.47545,160.4980,0.828007,3.735860,...,-184.13200,7.901370,1.70644,-0.494699,-2.058300,0.819184,0.439152,2.36470,1.14383,3
2,2,0.317816,19,-0.432571,-0.382644,1383.2600,19.71290,31.1026,-0.515354,34.430800,...,7.43721,37.218100,3.25339,0.337934,0.615037,2.216760,0.745268,1.69679,12.30550,6
3,3,0.210753,17,-0.616454,0.946362,-119.2530,4.08235,185.2570,1.383310,-47.521400,...,9.66778,0.626942,1.49425,0.517513,-10.222100,2.627310,0.617270,1.45645,10.02880,2
4,4,0.439671,20,0.968126,-0.092546,74.3020,12.30650,72.1860,-0.233964,24.399100,...,290.65700,15.604300,1.73557,-0.476668,1.390190,2.195740,0.826987,1.78485,7.07197,1


In [4]:
test_data = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
test_data.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,250000,0.812665,15,-1.239120,-0.893251,295.5770,15.87120,23.04360,0.942256,29.898000,...,0.446389,-422.332,-1.44630,1.69075,1.059300,-3.010570,1.94664,0.529470,1.386950,8.78767
1,250001,0.190344,131,-0.501361,0.801921,64.8866,3.09703,344.80500,0.807194,38.421900,...,0.377179,10352.200,21.06270,1.84351,0.251895,4.440570,1.90309,0.248534,0.863881,11.79390
2,250002,0.919671,19,-0.057382,0.901419,11961.2000,16.39650,273.24000,-0.003300,37.940000,...,0.990140,3224.020,-2.25287,1.55100,-0.559157,17.838600,1.83385,0.931796,2.336870,9.05400
3,250003,0.860985,19,-0.549509,0.471799,7501.6000,2.80698,71.08170,0.792136,0.395235,...,1.396880,9689.760,14.77150,1.41390,0.329272,0.802437,2.23251,0.893348,1.359470,4.84833
4,250004,0.313229,89,0.588509,0.167705,2931.2600,4.34986,1.57187,1.118300,7.754630,...,0.862502,2693.350,44.18050,1.58020,-0.191021,26.253000,2.68238,0.361923,1.532800,3.70660


In [5]:
samp_sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
samp_sub.head()

,id,loss
0,250000,0
1,250001,0
2,250002,0
3,250003,0
4,250004,0


# Parameter

In [6]:
lgb_params = {
    'metric': 'RMSE',
    'lambda_l1': 1e-07, 
    'lambda_l2': 1e-07, 
    'num_leaves': 31, 
    'feature_fraction': 0.8, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 3, 
    'min_child_samples': 19,
    'num_threads': 4
}

In [7]:
cb_params = {
    'num_trees': 1000, 
    'od_wait': 1200, 
    'learning_rate': 0.1, 
    'l2_leaf_reg': 64, 
    'subsample': 0.83, 
    'random_strength': 17.17, 
    'max_depth': 16, 
    'min_data_in_leaf': 20, 
    'leaf_estimation_iterations': 3,
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'bootstrap_type': 'Bernoulli',
    'leaf_estimation_method': 'Newton',
    'random_seed': 42,
    "thread_count": 4
}

# Train

In [8]:
automl = TabularAutoML(task = Task('reg', ),
                       timeout = 3600,
                       cpu_limit = 4,
                       reader_params = {'n_jobs': 4, 'cv': 10, 'random_state': 42},
                       general_params = {'use_algos': [['lgb', 'cb']]},
                       lgb_params = {'default_params': lgb_params, 'freeze_defaults': True},
                       cb_params = {'default_params': cb_params, 'freeze_defaults': True},
                       verbose = 2
                      )

pred = automl.fit_predict(train_data, roles = {'target': 'loss', 'drop': ['id']})

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


Start automl preset with listed constraints:
- time: 3600 seconds
- cpus: 4 cores
- memory: 16 gb

Train data shape: (250000, 102)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 3555.799206018448 secs
Start fitting Selector_LightGBM ...

===== Start working with fold 0 for Selector_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's l2: 62.5172
[200]	valid's l2: 62.2844
[300]	valid's l2: 62.255
[400]	valid's l2: 62.2481
Early stopping, best iteration is:
[327]	valid's l2: 62.2324
Selector_LightGBM fitting and predicting completed
Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...

===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====

Training until validation scores don't improve for 100 rounds
[100]	valid's l2: 62.5233
[200]	valid's l2: 62.2446
[300]	valid's l2: 62.1996
[400]	valid's l2: 62.199
Early stopping, best iteration is:
[349]	valid's l2: 62.1746

===== Start working with fold

Time limit exceeded after calculating fold 6


Lvl_0_Pipe_0_Mod_1_CatBoost fitting and predicting completed
Time left -4.289296388626099


Time limit exceeded. Last level models will be blended and unused pipelines will be pruned.


Blending: Optimization starts with equal weights and score -61.84571189533628
Blending, iter 0: score = -61.663040741569525, weights = [1. 0.]
Blending, iter 1: score = -61.663040741569525, weights = [1. 0.]
No score update. Terminated

Automl preset training completed in 3604.51 seconds.


# Predict

In [9]:
test_pred = automl.predict(test_data)

# Submission

In [10]:
samp_sub['loss'] = test_pred.data[:, 0]
samp_sub.to_csv('submission.csv', index = False)